In [2]:
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
import librosa
import tf2onnx

import pandas as pd

2025-08-04 14:47:25.218587: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754344045.366960 1700178 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754344045.397238 1700178 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754344045.738645 1700178 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754344045.738680 1700178 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754344045.738684 1700178 computation_placer.cc:177] computation placer alr

In [15]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
# Load the model.
model = hub.KerasLayer('https://www.kaggle.com/models/google/surfperch/TensorFlow2/1/1')

# load the data

data = pd.read_csv('/home/a.jajodia.229/acoustic/acoustic_knowledge_discovery/perch_testing/paola.csv')



Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x77143a435150>>
Traceback (most recent call last):
  File "/home/a.jajodia.229/acoustic/acoustic_knowledge_discovery/.venv/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [10]:
model = tf.saved_model.load('/home/a.jajodia.229/acoustic/acoustic_knowledge_discovery/surfperch_export')

In [ ]:
onnx_model, _ = tf2onnx.convert.from_keras(model,opset=13)

AttributeError: '_UserObject' object has no attribute 'output_names'

In [1]:
data

NameError: name 'data' is not defined

In [17]:
data['dataset'].value_counts()
data['dataset'] = np.where(data['filepath'].str.contains('Williams'), 'Williams', 'Paola')

In [18]:
labels = ["anthrop", "anthrop_boat_engine", "anthrop_bomb", "anthrop_mechanical", "bioph", "bioph_cascading_saw", "bioph_chatter", "bioph_chorus", "bioph_crackle", "bioph_croak", "bioph_damselfish", "bioph_dolphin", "bioph_double_pulse", "bioph_echinidae", "bioph_epigut", "bioph_grazing", "bioph_grouper_a", "bioph_grouper_groan", "bioph_growl", "bioph_holocentrus", "bioph_knock", "bioph_knock_croak_a", "bioph_knock_croak_b", "bioph_knock_croak_c", "bioph_low_growl", "bioph_megnov", "bioph_midshipman", "bioph_mycbon", "bioph_pomamb", "bioph_pulse", "bioph_rattle", "bioph_rattle_response", "bioph_series_a", "bioph_series_b", "bioph_stridulation", "bioph_whup", "geoph", "geoph_waves"]

In [32]:
from tqdm import tqdm

result = []


for filepath in tqdm(data['filepath']):
    waveform, sample_rate = tf.audio.decode_wav(tf.io.read_file(filepath))
    if waveform.shape[0] == 0:
        continue
    waveform = tf.squeeze(waveform)
    
    waveform = librosa.resample(waveform.numpy(), orig_sr=int(sample_rate), target_sr=32000)
    
    waveforms = tf.split(
                    waveform, [160000 for _ in range(waveform.shape[0] // 160000)] + [waveform.shape[0] % 160000], axis=0
                )
    
    sounds = []
    
    
    if 'Paola' in filepath:
        
        agg_labels = tf.zeros(38)
        
        for i in range(len(waveforms)-1):
            output = model.infer_tf(waveforms[i][np.newaxis, :])

            reef_labels = output['reef_label'][0]
            
            agg_labels += reef_labels
            
        sig_mask = tf.argmax(agg_labels)
        
        result_data = {
        "filepath": filepath,
        "sounds": labels[int(sig_mask)]
        }
        result.append(result_data)
    else:
        for i in range(len(waveforms)-1):
            output = model.infer_tf(waveforms[i][np.newaxis, :])

            reef_labels = output['reef_label'][0]
            
            sig_mask = tf.argmax(reef_labels)
            
            result_data = {
            "filepath": filepath,
            "sounds": labels[int(sig_mask)]
            }
            result.append(result_data)
    

100%|██████████| 7070/7070 [13:29<00:00,  8.73it/s]


In [33]:
labels_df = data[['labels', 'filepath']]

In [34]:
record = []

for res in result:
    if res is not None:
        record_data = {
            'filepath': res['filepath'],
            'sound': res['sounds']
        }
        record.append(record_data)
pd.DataFrame.from_records(record).merge(labels_df, how='left', on='filepath').to_csv('surfperch_results.csv')

[{'filepath': '/home/a.jajodia.229/acoustic/local_data/coral/Paola/Degraded_Reef/July 2024/20240723_191200.WAV',
  'sound': 'bioph_cascading_saw'},
 {'filepath': '/home/a.jajodia.229/acoustic/local_data/coral/Paola/Non_Degraded_Reef/June 2025/20250624_144120.WAV',
  'sound': 'bioph_cascading_saw'},
 {'filepath': '/home/a.jajodia.229/acoustic/local_data/coral/Paola/Degraded_Reef/May 2025/20250322_190300.WAV',
  'sound': 'bioph_cascading_saw'},
 {'filepath': '/home/a.jajodia.229/acoustic/local_data/coral/Paola/Non_Degraded_Reef/June 2025/20250625_145520.WAV',
  'sound': 'bioph_cascading_saw'},
 {'filepath': '/home/a.jajodia.229/acoustic/local_data/coral/Paola/Degraded_Reef/May 2025/20250320_070403.WAV',
  'sound': 'bioph_cascading_saw'},
 {'filepath': '/home/a.jajodia.229/acoustic/local_data/coral/Paola/Degraded_Reef/May 2025/20250319_060000.WAV',
  'sound': 'bioph_cascading_saw'},
 {'filepath': '/home/a.jajodia.229/acoustic/local_data/coral/Paola/Degraded_Reef/May 2025/20250320_192600.W